In [186]:
!pip install langchain langchain-community openai requests python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports and setup

In [187]:
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "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"

# Load the tool
This example uses the NASA `AstronomyPictureOfTheDayTool` to fetch the Astronomy Picture Of The Day.

In [188]:
import sys, os
import inspect
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from gofannon.nasa.apod import AstronomyPhotoOfTheDayTool

apod_instance = AstronomyPhotoOfTheDayTool(api_key="200z6QIxgOePkdCP8Ip347mSJKQ737PaLda85mke")


def apod_wrapper(input_str: str = "") -> str:
    return apod_instance.fn()

apod_tool = Tool(
    name="AstronomyPhotoOfTheDayTool",
    func=apod_wrapper,
    description=apod_instance.definition["function"]["description"]
)

# Initializing the LangChain agent

In [189]:
llm = ChatOpenAI(
    model_name="mistralai/mistral-7b-instruct",
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1"
)

In [190]:
# Initialize OpenAI chat model
# llm = ChatOpenAI(temperature=0)

# Register the agent with the tool
agent = initialize_agent(
    tools=[apod_tool],
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)

# Running a Query


In [191]:
query = "What is the Astronomy Photo of the Day?"
response = agent.run(query)

print(response)



> Entering new AgentExecutor chain...
 To find the Astronomy Picture of the Day, I can use the `AstronomyPhotoOfTheDayTool`.

Action: AstronomyPhotoOfTheDayTool
Action Input: ''
Observation: {'title': "The Gargoyles' Eclipse", 'date': '2025-04-05', 'explanation': "In dramatic silhouette against a cloudy daytime sky over Paris, France, gargoyles cast their monstrous gaze outward from the west facade of Notre Dame Cathedral.  Taken on March 29, this telephoto snapshot also captures the dramatic silhouette of a New Moon against the bright solar disk in a partial solar eclipse. Happening high in Parisian skies, the partial eclipse was close to its maximum phase of about 23 percent. Occurring near the end of the first eclipse season of 2025, this partial solar eclipse followed the total eclipse of the Full Moon on March 13/14. The upcoming second eclipse season of 2025 will see a total lunar eclipse on September 7/8 and partial solar eclipse on September 21. The partial solar eclipse will